<a href="https://colab.research.google.com/github/navilindo/IUA/blob/main/nmt_feb_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 
import string

In [2]:
# using pandas to reas and open data
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nlp-data/short/arabic_english.txt",delimiter="\t",names=["eng","ar"])
df1

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [3]:
arabic = df1['ar']
arabic

0                                                  مرحبًا.
1                                                    اركض!
2                                                  النجدة!
3                                                    اقفز!
4                                                      قف!
                               ...                        
24633    شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634    استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635    تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636    للمشاركة في التحدي اتبع الخطوات الموضحة على ال...
24637    ستجد ايضا روابط لمجموعة من منصات ابداع الميم ا...
Name: ar, Length: 24638, dtype: object

In [4]:
english = df1['eng']
english

0                                                      Hi.
1                                                     Run!
2                                                    Help!
3                                                    Jump!
4                                                    Stop!
                               ...                        
24633    rising voices promoting a more linguistically ...
24634    following last year s successful campaign we i...
24635    during last year s challenge we also met langu...
24636    to take part just follow the simple steps outl...
24637    you will also find links to some free web base...
Name: eng, Length: 24638, dtype: object

In [5]:
df1.shape

(24638, 2)

In [6]:
##
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [8]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [15]:
num_sentences = 2000 #for test we put small number but we will change it to 20k

# beginning of data loading
encoder_inputs = []
encoder_outputs = []
decoder_inputs = []

In [16]:
count = 0
for line in arabic:
  count += 1
  if count > num_sentences:
        break
  input_line = line
  encoder_inputs.append(input_line)

In [17]:
count = 0
for line in english:
  count += 1
  if count > num_sentences:
        break
  output_line = line
  encoder_outputs.append(output_line)
  decoder_inputs.append(line)

In [18]:
## check the encoder inputs, outputs and decoder inputs

print("sample encoder inputs : ", len(encoder_inputs))
print("sample encoder outputs : ", len(encoder_outputs))
print("sample decoder inputs: ", len(decoder_inputs))

sample encoder inputs :  2000
sample encoder outputs :  2000
sample decoder inputs:  2000


In [19]:
# test if the sentences are well loaded..

print(encoder_inputs[1800])
print(encoder_outputs[1800])
print(decoder_inputs[14])

و كيف أجبت؟
How did you reply?
Cheers!


In [20]:
# cleaning Arabic sentences 
arab_sents = []

for in_sent in encoder_inputs:
  sent = clean_sentence(in_sent)
  arab_sents.append(sent)

In [21]:
# cleaning english sentences
eng_sents = []

for out_sent in encoder_outputs:
  sent = clean_sentence(out_sent)
  eng_sents.append(sent)

In [22]:
# tokenizing sentences into words
arab_tokens, arab_tokenizer = tokenize(arab_sents)
eng_tokens, eng_tokenizer = tokenize(eng_sents) 

print('Maximum length of Arabic sentence: {}'.format(len(max(arab_tokens,key=len))))
print('Maximum length of English sentence: {}'.format(len(max(eng_tokens,key=len))))

Maximum length of Arabic sentence: 7
Maximum length of English sentence: 5


In [23]:
# unique language vocabulary

arab_vocab = len(arab_tokenizer.word_index) + 1
eng_vocab = len(eng_tokenizer.word_index) + 1

print("The Arabic vocabulary is of {} unique words".format(arab_vocab))
print("The English vocabulary is of {} unique words".format(eng_vocab))

The Arabic vocabulary is of 2099 unique words
The English vocabulary is of 995 unique words


In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# padding to produce same length of encoder input sentences

arab_max_len = int(len(max(arab_tokens, key=len)))
eng_max_len = int(len(max(eng_tokens, key=len)))

arab_padded_sentences = pad_sequences(arab_tokens, arab_max_len, padding = "post")
eng_padded_sentences = pad_sequences(eng_tokens, eng_max_len, padding = "post")

In [25]:
# Reshaping data

arab_padded_sentences = arab_padded_sentences.reshape(*arab_padded_sentences.shape, 1)
eng_padded_sentences = eng_padded_sentences.reshape(*eng_padded_sentences.shape, 1)

In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding

In [27]:
# embedding our input layer

input_sequence = Input(shape=(arab_max_len,))
embedding = Embedding(input_dim=arab_vocab, output_dim=128,)(input_sequence)

In [29]:
# optimizers and losses

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy